In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import os
import pandas as pd
import numpy as np
import conda_scripts
import forward_run
import matplotlib.pyplot as plt
import conda_scripts.plot_help as ph
from conda_scripts import sv_budget
import matplotlib as mpl
import numpy as np
import flopy
import geopandas as gpd

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
main = 'postprocess'
fold =  'pestaws_v1'

out_folder = os.path.join(main, fold)
if not os.path.exists(out_folder):
    os.makedirs(out_folder, exist_ok =True)

In [ ]:
ml = conda_scripts.sv_budget.load_sv_model.get_model(workspace)

In [ ]:
workspace

In [ ]:
import flopy.modflow.mfhob as hfb
import flopy.modflow.mfhyd as hyd

In [ ]:
df = pd.read_csv(r"C:\GSP\waterlevel\GIS\hydro_experiment\hydros__v3_SON_allmodmonths_to_20240808\hydros_predicted_SON.csv")

In [ ]:
df = df.query('`Station Name`.str.contains("_mod") == False', engine='python')

In [ ]:
df = df.drop_duplicates(['Station Name', 'datetime'])

In [ ]:
df.loc[:,'Depth'] = "Other"
df.loc[df.Shallow,'Depth'] = "Shallow"
df.loc[df.Deep,'Depth'] = "Deep"

df = df.loc[~(df.Depth=='Other')]
df.drop_duplicates('Station Name').Depth.value_counts()



In [ ]:
lrc = df.drop_duplicates('Station Name').set_index('Station Name').apply( lambda row: ml.modelgrid.intersect(row['Easting'], row['Northing']),axis = 1)

lrc = pd.DataFrame(lrc.to_list(), index = lrc.index, columns=['i','j'])

In [ ]:
df = pd.merge(df, lrc, left_on = 'Station Name', right_index = True)

In [ ]:
df.query("Deep==True | Shallow==True").columns

In [ ]:
import cartopy.crs as ccrs
import contextily as ctx

In [ ]:
z = np.genfromtxt(os.path.join(ml.model_ws, 'model_arrays', 'zonation_3.csv'), delimiter = ' ')

zotther = z.copy()
zotther[zotther>8] = 0

zones = {i:zotther  for i in range(7) }
zones[1] = z
plt.imshow(zones[1])

z = conda_scripts.arich_functions.array2rc(zones[1],'zone').astype({'zone':int})

aliases = {1: 'Bay', 2: 'EastSide', 3: 'SouthCent', 4: 'Kenwood', 5: 'VOM', 6: 'AguaCal',7:'WestSide',8:'CitySon',9:'Highlands'}
z.loc[:,'zone'] =z.loc[:,'zone'].replace(aliases)
z = z.query("zone!=0")

geoms = conda_scripts.arich_functions.get_model_shp(ml.modelgrid, 2226).drop(columns = ['row','col'])

z = gpd.GeoDataFrame(pd.merge(z, geoms, on = ['i','j']))
zplot = z.dissolve('zone').reset_index()

In [ ]:
vcounts = pd.read_csv(r"C:\GSP\waterlevel\GIS\hydro_experiment\hydros__v3_SON_allmodmonths_to_20240808\seasinfo_w_predicted_SON.csv")
vcounts = vcounts.loc[:,'Station Name'].value_counts().to_frame("num_meas")
vcounts = vcounts.loc[~vcounts.index.str.contains('mod')]

In [ ]:
vcounts

In [ ]:
dfinfo = pd.merge(df.loc[:,['Station Name', 
       'Site', 'Easting', 'Northing', 'Latitude',
       'Longitude',  'rasterelevation', 'slope',
       'Simple_Bou', 'Complete_B', 'isostatic',  'Geol_Krig',
 
      'Depth', 'i', 'j', 
       ]].drop_duplicates('Station Name'),
                  z, on = ['i','j'], how = 'left')
dfinfo = dfinfo.drop(columns = ['geometry','row','column'])

dfinfo = pd.merge(dfinfo, vcounts,on = 'Station Name' )



# make list of important wells for PEST calibration

In [ ]:
#these 

important = ['son0328',#eastside
            'son0065', #aguacal deep
             'son0069', #aguacal shallow
             'son0096', #westside deep
            ]

imp = dfinfo.loc[dfinfo.loc[dfinfo.loc[:,'Station Name'].str.lower().isin(important)].groupby(['zone', 'Depth'])['num_meas'].idxmax()]
imp

In [ ]:
important_hydros = dfinfo.loc[dfinfo.groupby(['zone', 'Depth'])['num_meas'].idxmax()]



#concat 2 dfs except for rows in the second that have the same values in Depth 	and  	zone columns. both values in the second df need to match the first df to be removed

# Identify rows in df2 to keep
df2_filtered = important_hydros[~important_hydros[['Depth', 'zone']].apply(tuple, axis=1).isin(imp[['Depth', 'zone']].apply(tuple, axis=1))]

# Concatenate df1 with the filtered df2
important_hydros = pd.concat([imp, df2_filtered], ignore_index=True)

important_hydros = gpd.GeoDataFrame(important_hydros, geometry = gpd.points_from_xy(important_hydros.Easting, important_hydros.Northing), crs = 2226)

important_hydros = important_hydros.query("~ zone.str.contains('High')")

In [ ]:
important_hydros

In [ ]:
important_hydros_folder = os.path.join(os.getcwd(), '..','waterlevel','important_hydros')

os.makedirs(important_hydros_folder, exist_ok=True)

In [ ]:
plt.figure(figsize = (6,8), dpi = 250)
ax = plt.subplot(1,1,1, projection = ccrs.epsg(2226))
zplot.exterior.plot(ax = ax,  )
zplot.plot('zone', ax = ax, alpha =.5  )
important_hydros.plot('Depth', ec = 'k', ax = ax, cmap = 'magma', legend = True)
conda_scripts.plot_help.label_points(ax,important_hydros, 'Station Name', text_color='k') 
conda_scripts.arich_functions.add_basemaps(ax)
ax.set_title('map_of_very_important_wells_for_pest'.replace('_',' ').title())
plt.savefig(os.path.join(important_hydros_folder,'map_of_very_important_wells_for_pest.png'), dpi = 250)

In [ ]:
important_hydros.to_file(os.path.join(important_hydros_folder, 'very_important_wells.shp'))

In [ ]:
important_hydros_folder

In [ ]:
important_hydros

In [ ]:
important_hydros.loc[:,'stat'] = important_hydros.loc[:,'Station Name'].str.lower()
df.loc[:,'stat'] = df.loc[:,'Station Name'].str.lower()

# make individual plots

In [ ]:
for stat, group in important_hydros.groupby('Station Name'):

    row = df.query(f"stat.str.lower().str.contains('{stat.lower()}')").copy()
    row.loc[:,'date'] = pd.to_datetime(row.datetime)
    rowind = row['i'].unique()[0]
    column = row['j'].unique()[0]
    
    x = row['Easting'].unique()[0]
    y = row['Northing'].unique()[0]
    fig, ax = helpers.make_plot('',x,y)


    row.plot(x = 'date', y = 'predicted',ax = ax)
    # res.loc[:,stat].rename("GWLE").plot(ax = ax, label = "GWLE")
    # out.loc[:,stat].plot(ax = ax ,label = "OWHM")
    # out.loc[:,stat].head(1).plot(ax = ax ,c = 'r', marker = 'o', label = "Starting")
    ax.legend()
    
    # lays = row.iloc[[0],:].reset_index().at[0,'mlays']
    # lays = ', '.join([str(j+1) for j in lays])
    ax.set_title( f"{stat} {group['zone'].unique()[0]}, {group['Depth'].unique()[0]}" )

    ph.yrange_(ax)
    ax.grid(True)
    plt.savefig(os.path.join(important_hydros_folder, stat+'.png' ))
    # print(stat, rowind, column, row.shape,os.path.abspath(os.path.join(out_folder,'hydros', stat+'.png' )))
    plt.savefig(os.path.join(important_hydros_folder, stat+'.png' ))
    # plt.close(fig)
    
    # .plot(x = 'datetime', y = 'predicted', title = stat)

In [ ]:
import importlib
import helpers
from shapely.geometry import Point
importlib.reload(helpers)

In [ ]:
for statgroup, zonegroup in important_hydros.groupby('zone'):
    v = f"{zonegroup.loc[:,'Station Name'].unique()[0].lower()}"
    row = df.query(f"stat.str.lower().str.contains('{v}')").copy()

    rowind = row['i'].unique()[0]
    column = row['j'].unique()[0]
    
    x = row['Easting'].unique()[0]
    y = row['Northing'].unique()[0]
    fig, ax,ax3 = helpers.make_plot('',x,y,return_plot_ax=True)

    colors = {'deep':'b',
             'shallow':'orange'}
    for stat, group in zonegroup.groupby('Station Name'):
        row = df.query(f"stat.str.lower().str.contains('{stat.lower()}')").copy()
        rowind = row['i'].unique()[0]
        column = row['j'].unique()[0]
        
        c = colors[group['Depth'].unique()[0].lower()]
        
        row.loc[:,'date'] = pd.to_datetime(row.datetime)   
        row.plot(x = 'date', y = 'predicted',ax = ax, label = group['Depth'].unique()[0]+' '+stat, color = c)
        
        ax.set_title( f"{', '.join(zonegroup.loc[:,'Station Name'].values)} {group['zone'].unique()[0]}" )
        ax.grid(True)

        x = row['Easting'].unique()[0]
        y = row['Northing'].unique()[0]
        
        pt = gpd.GeoSeries(Point(x, y), crs = 2226)   
        pt.plot(ax = ax3,  markersize = 40,marker = '*', color = c, edgecolor = c)
        

        plt.savefig(os.path.join(important_hydros_folder, statgroup+'.png' ))
        
    ax.legend(loc = 'upper left', bbox_to_anchor = (1,1))